In [1]:
import os

In [2]:
model_parent_path = os.getcwd()
model_path = 'google-t5/old_t5-large_qasc_epoch_0_.pkl'
model_name = None
tokenizer_name = "google-t5/t5-large"
max_answer_length = 300
BATCH_SIZE = 10
accumulate_step = None
DATA_NAME_SINGLE = "qasc"
NUM_EPOCHS = 1
VISIBLE_DEVICE = "0,1,2"
DEVICE = 0

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = VISIBLE_DEVICE  # cannot work
print(
    f"Loi params: {model_path}{'#'*5}{max_answer_length}{'#'*5}batch_size={BATCH_SIZE};accumulate_step={accumulate_step}"
)
DATABASE_NAME = [DATA_NAME_SINGLE]  # ,'arc_hard']#,'race','mctest',]

Loi params: google-t5/old_t5-large_qasc_epoch_0_.pkl#####300#####batch_size=10;accumulate_step=None


In [4]:
import torch
import pickle
from torch.utils.data import Dataset, DataLoader
from tqdm import trange
from transformers import T5Tokenizer, T5ForConditionalGeneration, Adafactor
from functools import wraps, partial
from torch.nn.modules.sparse import Embedding
from torch.optim import Adam, SGD
import torch.nn as nn
from tqdm import tqdm
from random_utils import set_seed
import pandas as pd

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [5]:
# model_name = "loi_with_padding_just_same_answer_len_70.pkl"

# "loi_with_padding_1.pkl"#
# model_name = (
#     "allenai/unifiedqa-v2-t5-large-1363200"  # you can specify the model size here
# )
tokenizer = T5Tokenizer.from_pretrained(tokenizer_name)

model_original = (
    T5ForConditionalGeneration.from_pretrained(
        f"{model_parent_path}/{model_path}", device_map="auto"
    )
    if model_path
    else T5ForConditionalGeneration.from_pretrained(model_name, device_map="auto")
)
model = model_original
# model.hf_device_map

/home/ldh0033@auburn.edu/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:246: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on google-t5/t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes

In [6]:
dataset_test = []
[dataset_test.append([]) for x in DATABASE_NAME]
moi_index = []

for i, dataname in enumerate(DATABASE_NAME):
    dataset_test[i] = pickle.load(
        open(f"multiple_choice_datasets/{dataname}_test.pkl", "rb")
    )

In [7]:
import textwrap

# ques=4
print(textwrap.fill(dataset_test[0][0][0]))
# print('correct: ',dataset_test[ques][1],'length',len(dataset_test[0][0]))

Climate is generally described in terms of local weather conditions.
Climate is generally described in terms of what? \n ( ) sand ( )
occurs over a wide range ( ) forests ( ) Global warming ( ) rapid
changes occur ( ) local weather conditions ( ) measure of motion ( )
city life


In [8]:
def hook(hook_before, oldfunc, hook_after):

    def foo(*args, **kwargs):
        hook_before(*args, **kwargs)
        aa = oldfunc(*args, **kwargs)
        hook_after(*args, **kwargs)
        return aa

    return foo

In [9]:
def new_compute_bias(self, query_length, key_length, device=None):
    """Compute binned relative position bias"""
    if device is None:
        device = self.relative_attention_bias.weight.device
    context_position = torch.arange(query_length, dtype=torch.long, device=device)[
        :, None
    ]
    memory_position = torch.arange(key_length, dtype=torch.long, device=device)[None, :]

    relative_position = (
        memory_position - context_position
    )  # shape (query_length, key_length)
    if self.is_decoder:
        values = values.permute([2, 0, 1]).unsqueeze(
            0
        )  # shape (1, num_heads, query_length, key_length)
        return values

    anchors = self.anchor_array
    values = []
    for anchor in anchors:
        mot = [[anchor[idx], anchor[idx + 1]] for idx in range(len(anchor) - 1)]
        max_answer_length = max([x[1] - x[0] for x in mot])
        # print(a, b, c, d, max_answer_length)
        context_position_new = context_position.clone()
        for i in range(1, len(mot)):
            context_position_new[mot[i][0] : mot[i][1]] = (
                context_position_new[mot[i][0] : mot[i][1]] - mot[0][0]
            )
        context_position_new[-1] = mot[0][0] + 2 * max_answer_length
        memory_position_new = context_position_new.clone().view(1, -1)
        relative_position = (
            memory_position_new - context_position_new
        )  # shape (query_length, key_length)
        for i in range(len(mot)):
            for j in range(len(mot)):
                if i != j:
                    x = mot[i]
                    y = mot[j]
                    relative_position[x[0] : x[1], y[0] : y[1]] += max_answer_length
        relative_position_bucket = self._relative_position_bucket(
            relative_position,  # shape (query_length, key_length)
            bidirectional=(not self.is_decoder),
            num_buckets=self.relative_attention_num_buckets,
            max_distance=self.relative_attention_max_distance,
        )
        value = self.relative_attention_bias(relative_position_bucket)
        values.append(value)
    values = torch.stack(values)  # shape [1, 91, 91, 16]
    values = values.permute(
        [0, 3, 1, 2]
    )  # shape (batch size, num_heads, query_length, key_length)
    return values

In [10]:
import textwrap


extra_dim_learning = []


def set_mode(MODE):
    itself = model.encoder.block[0].layer[0].SelfAttention
    if MODE == "new":
        # itself.forward = partial(modified_self_attention_forward, itself)
        itself.compute_bias = partial(new_compute_bias, itself)
        model.forward = hook(
            input_before_hooker,
            partial(T5ForConditionalGeneration.forward, model),
            input_after_hooker,
        )

    else:
        # itself.forward = partial(
        #     model.encoder.block[0].layer[0].SelfAttention.__class__.forward, itself
        # )
        itself.compute_bias = partial(
            model.encoder.block[0].layer[0].SelfAttention.__class__.compute_bias, itself
        )
        model.forward = T5ForConditionalGeneration.forward


def check_encoded(all_input_ids):
    anchors = []
    for input_ids in all_input_ids:
        # print('\n'.join([f'{x.item()},{y}' for x,y in zip(input_ids, tokens)][50:]))
        original = input_ids.tolist()
        anchor = []
        for i in range(len(input_ids)):
            if (
                i < len(input_ids) - 2
                and input_ids[i] == 41
                and input_ids[i + 1] == 3
                and input_ids[i + 2] == 61
            ) or original[i] == 1:
                anchor.append(i)
        anchors.append(anchor)
    return anchors


def input_before_hooker(*args, **kwargs):
    input_ids = kwargs["input_ids"]
    # print('old ',input_ids)
    anchors = check_encoded(input_ids)
    final_inputs = []
    for input_id, anchor in zip(input_ids, anchors):
        input_id = input_id.tolist()

        real_max_len = max(
            [anchor[idx + 1] - anchor[idx] for idx in range(len(anchor) - 1)]
        )
        if real_max_len > max_answer_length:
            print(f"ALERT: MAX LENGTH IS {real_max_len}")
        for x in reversed(range(1, len(anchor))):
            if anchor[x] - anchor[x - 1] < max_answer_length:
                [
                    input_id.insert(anchor[x], 0)
                    for _ in range(max_answer_length - (anchor[x] - anchor[x - 1]))
                ]

        final_inputs.append(input_id)

    max_length = max([len(input) for input in final_inputs])
    mask = [[1] * max_length] * len(final_inputs)
    for idx, input in enumerate(final_inputs):
        for x in range(max_length):
            if x >= len(input):
                mask[idx][x] = 0
        for x in range(max_length - len(input)):
            input.append(0)
    kwargs["input_ids"] = torch.tensor(final_inputs).to(input_ids.device)
    kwargs["attention_mask"] = torch.tensor(mask).to(input_ids.device)
    # print('new ',kwargs["input_ids"])
    anchors = check_encoded(kwargs["input_ids"])
    model.encoder.block[0].layer[0].SelfAttention.anchor_array = anchors


def input_after_hooker(*args, **kwargs):
    model.encoder.block[0].layer[0].SelfAttention.anchor_array = None


# set_mode('old')
set_mode("new")
# run_model([dataset_train[0][0],dataset_train[1][0]])
# run_model(["""A person wants to start""", 'mot hai ba'])

In [11]:
def measure_unalike(arr, print_arr=False):
    n = len(arr)
    arr = pd.Series(arr).value_counts()
    if print_arr:
        print(arr)
    return 1 - ((arr / n) ** 2).sum()


question_to_do = 5
per_question = 20


def get_model_forward(input_ids, attention_mask, model=model):
    with torch.no_grad():
        start = []
        [start.append([0]) for x in range(len(input_ids))]
        for k in range(max_answer_length):
            # print(torch.tensor(start).shape)
            result = model(
                input_ids=input_ids.to(DEVICE),
                attention_mask=attention_mask.to(DEVICE),
                decoder_input_ids=torch.tensor(start).to(DEVICE),
                output_attentions=True,
            )
            item = result.logits.argmax(dim=2)[:, -1]
            # print('loi',result.logits.shape, item)
            for index in range(len(item)):
                start[index].append(item[index].item())
            if torch.allclose(item, torch.tensor(1)):
                break
            #     break
    result = []
    for batch in start:
        y = -1
        for index, x in enumerate(batch):
            if x == 1:
                y = index
                break
        result.append(batch[: y + 1] if y > -1 else batch)
    return [tokenizer.decode(x, skip_special_tokens=True) for x in result]


def run_model(input_strs):
    if input_strs is str:
        input_strs = [input_strs]
    input_ids_wrapper = tokenizer(input_strs, padding=True, return_tensors="pt")

    answer = get_model_forward(
        input_ids_wrapper["input_ids"], input_ids_wrapper["attention_mask"]
    )
    return answer

In [12]:
kk = [
    (index, x, y)
    for index, (x, y) in enumerate(model.named_parameters())
    if y.requires_grad == True
]
[(index, x) for index, x, y in kk if "decoder" in x]
len(kk)
all_position_weight = [
    y
    for index, x, y in kk
    if ("extra_dimension_embedding" in x)
    or (("encoder" in x) and ("relative_attention_bias" in x))
]
to_train_model = [y for index, x, y in kk]

In [13]:
# [(print(idx),CheckTransform.__call__(None, data_array[idx])) for idx,x in enumerate(data_array)]

In [16]:
def evaluate():
    ll = 1
    wrong_answers = [[] for x in range(ll)]
    got_2 = [[] for x in range(ll)]
    got_1 = [[] for x in range(ll)]
    answers = [[] for x in range(ll)]
    last_str = None
    last_acc = None
    for ix in range(ll):
        print(f"Name {DATABASE_NAME[ix]}")
        # [pickle.load(open(f"multiple_choice_datasets/obqa_fact_test.pkl", "rb"))]:
        # print(f"test {data==dataset_test}")
        count = 0
        count1 = 0
        count2 = 0
        count10 = 0
        total = 0
        data = dataset_test[ix]
        pbar1 = trange(len(data))
        for ques in pbar1:
            question = data[ques][0]
            key = data[ques][1]
            total += 1
            answer = run_model(question)[0]
            answers[ix].append(answer)
            if key == answer:
                count += 1
            else:
                wrong_answers[ix].append(ques)
            if key[0] == answer[0]:
                count1 += 1
                got_1[ix].append(ques)
            if key[:2] == answer[:2]:
                count2 += 1
                got_2[ix].append(ques)
            if answer in question:
                count10 += 1
            last_str = f"{count}, {count1}, {count2}, {count10},{total},{count/total*100:.2f},{count10/total*100:.2f}"
            last_acc = f"{count/total*100:.2f}"
            pbar1.set_postfix_str(last_str)
    return last_str, last_acc,wrong_answers

In [17]:
_,_,wrong_answers= evaluate()

Name qasc


100%|██████████| 926/926 [05:55<00:00,  2.60it/s, 913, 920, 920, 923,926,98.60,99.68]  


### Measure resilient

In [ ]:
def measure_unalike(arr):
    n = len(arr)
    arr = pd.Series(arr).value_counts()
    return 1 - ((arr / n) ** 2).sum()


measure_unalike(["a", "a", "a"])

0.0